# Session 2.3: Embedding, Vektordatenbanken und RAG (Retrieval Augmented Generation)

## Quellen

- Huggingface MTEB (Massive Text Embedding Benchmark) Leaderboard: https://huggingface.co/spaces/mteb/leaderboard
    - Mit Auswahl Language 'deu' und Parameter 0-5000M -> *'multilingual-e5-large-instruct'* an erster Stelle
    - Was ist MTEB?: https://huggingface.co/blog/mteb#benchmark-your-model
    - *'multilingual-e5-large-instruct'* auf Ollama: https://ollama.com/jeffh/intfloat-multilingual-e5-large-instruct
- Bestenliste auf Ollama -> Models -> Embeddings jedoch *'nomic-embed-text'*
    - *'nomic-embed-text'* auf Ollama: https://ollama.com/library/nomic-embed-text

## Praxis

### 🛠️ Imports

In [ ]:
from aiworkshop_utils.standardlib_imports import os, json, base64, logging, Optional, List, pprint, shutil, glob
from aiworkshop_utils.thirdparty_imports import AutoTokenizer, load_dotenv, requests, BaseModel, Field, pd, cosine_similarity, plt, np, DataType, MilvusClient, tqdm
from aiworkshop_utils.custom_utils import show_pretty_json, encode_image, load_md_sections_from_path
from aiworkshop_utils.jupyter_imports import Markdown, HTML, widgets, display, JSON
from aiworkshop_utils.openai_imports import OpenAI
from aiworkshop_utils import config

### 💡 Konzept: Embedding

- Was ist Embedding?
    - Sprache (oder allgemein Daten) in Zahlen verwandeln
    - Wie wir schon gelernt haben, wird Text in Token-IDs umgewandelt, z.B. "Hi!" ergibt wie vorher zwei Token-IDs: [13347, 0]
    - Damit die Semantik eines Textes gedeutet werden kann, muss der Text eingebettet werden
    - Beispiel für ein 8D-Embedding: "Hi!" -> Token-IDs [13347, 0] -> Embedding als z.B. 8D-Vektor [0.12, -0.58, 0.33, 0.77, -0.41, 0.05, 0.99, -0.22]
    - Egal wie lange der Input ist - es wird immer ein Vektor mit gleicher Dimensionen-Anzahl erstellt
    - Die Bedeutung des Textes, nicht seine Länge oder Struktur werden eingebettet - Mehr Kontext im Text, bessere Einbettung
    - Vektoren gängiger Modelle haben viele Dimensionen: GPT-3 12.288, GPT-4 bis zu 16.000 oder darüber
    - Jede Dimension ist eine gelernte semantische Eigenschaft, die wir nur erahnen können, z.B. "Stimmung", "Humor", "Tier", ...
    - Je mehr Dimensionen, desto granularere semantische Feinunterscheidung
    - Daten müssen in den gleichen Vektorraum eingebettet werden, um die richtige semantische relative Distanz beizubehalten zwischen verschiedenen Daten
    - Ein Prompt, der auf Daten abzielt, muss im selben Vektorraum eingebettet werden, um die semantisch ähnlichsten Daten dazu richtig zu finden
    - Es gibt spezialisierte Embedding-Modelle, die eher gut in Embedding sind und nicht in der Generierung

- Was wird eigentlich eingebettet?
    - Levels: Word / Sentence / Document
    - Wort-Embeddings
        - Beispiel: ["König", "Königin", "Mann", "Frau"]
        - Methoden: Word2Vec, GloVe, FastText
        - Problem: Ein Wort gibt nichts über Kontext Preis -> Bank -> Sitzbank? Institution?
    - Satz-Embeddings
        - Beispiel: [„Die Bank ist bequem.“, „Ich gehe zur Bank, um Geld abzuheben.“]
        - Methoden: BERT, Sentence-BERT, Universal Sentence Encoder
    - Document-Embeddings
        - Beispiel: Zeitungsartikel
        - Methode: Oft wird ein Dokumentembedding aus Satz-Embeddings aggregiert (z. B. Mittelwert, Attention, oder mit speziellen Modellen wie Longformer etc.)

- Auf was ist zu achten?
    - Verschiedene Modelle haben verschiedene Dimensionen
        - Nomic hat z.B. 768 (12 Attention Heads x 64 Dimensionen pro Head)
        - BERT large hat 1024 (16 Attention Heads x 64 Dimensionen pro Head)
    - Die Vektordatenbank muss mit dieser Dimensions-Anzahl übereinstimmen
    - Multilingualität - bei Nomic vorhanden
    - Man kann über Python-Library sentence-transformer einbetten oder über Ollama Embedding gleich über API bereitstellen und auslagern - machen wir!

### Ollama Embedder

- Ollama Embedding Request
    - POST to "http://localhost:11434/api/embed"
    - requ -> model: name of model to generate embeddings from
    - requ -> input: text or list of text to generate embeddings for
    - opti -> truncate: truncates the end of each input to fit within context length.
        - Returns error if false and context length is exceeded. Defaults to true
    - opti -> options: additional model parameters listed in the documentation for the Modelfile such as temperature
    - opti -> keep_alive: controls how long the model will stay loaded into memory following the request (default: 5m)

- Ollama Embedding Response
    - 'model': '...'
    - 'embeddings': [[...,...,...,...]]
    - 'total_duration': '...'
    - 'load_duration': '...'
    - 'prompt_eval_count': '...'

- further sources:
    - ollama general: https://github.com/ollama/ollama?tab=readme-ov-file
    - ollama API: https://github.com/ollama/ollama/blob/main/docs/api.md#generate-embeddings
    - ollama Modelfile: https://github.com/ollama/ollama/blob/main/docs/modelfile.md

In [ ]:
class OllamaEmbedder:
    def __init__(self, url, model_name):
        self.url = url
        self.model_name = model_name

    def _post_request(self, texts):
        if isinstance(texts, str):
            texts = [texts]
        
        response = requests.post(
            self.url,
            json={
                "model": self.model_name,
                "input": texts
            }
        )
        return response.json()

    def get_embeddings(self, texts):
        response = self._post_request(texts)
        embeddings = response["embeddings"]
        # If only one embedding is returned, return it directly
        if isinstance(texts, str) or len(embeddings) == 1:
            return embeddings[0]
        return embeddings

    def get_full_response(self, texts):
        return self._post_request(texts)

In [ ]:
embedder_ollama = OllamaEmbedder(url=config.OAPI_EMBED_URL, model_name=config.OMODEL_NOMIC)

texts = ["Warum bin ich so fröhlich?", "Ich liebe Käse.", "Was ist künstliche Intelligenz?"]

fullresponse = embedder_ollama.get_full_response(texts)
show_pretty_json(fullresponse)

In [ ]:
vectors = embedder_ollama.get_embeddings(texts)

print(len(vectors))          # Anzahl der Texte
print(len(vectors[0]))       # Dimension des Embeddings
print(vectors[0][:10])       # Ersten 10 Werte

### OpenAISDK-Embedder

In [ ]:
# Openai SDK Client erstellen
client = OpenAI(
    base_url=config.OAPI_OPENAISDK_BASE_URL,
    api_key=config.OLLAMA_FAKE_API_KEY,  # fake key!
)

In [ ]:
class OpenaisdkEmbedder:
    def __init__(self, client, model_name):
        self.client = client
        self.model_name = model_name

    def _post_request(self, texts):
        if isinstance(texts, str):
            texts = [texts]
        
        # POST-Anfrage mit mehreren Texten gleichzeitig (Batch-Embedding)
        response = self.client.embeddings.create(
            model=self.model_name,
            input=texts,
            encoding_format="float"
            )

        return response
    
    def get_embeddings(self, texts):
        response = self._post_request(texts)
        embeddings = [data_item.embedding for data_item in response.data]
        # If only one embedding is returned, return it directly
        if isinstance(texts, str) or len(embeddings) == 1:
            return embeddings[0]
        return embeddings

    def get_full_response(self, texts):
        return self._post_request(texts)

In [ ]:
embedder_openaisdk = OpenaisdkEmbedder(client=client, model_name=config.OMODEL_NOMIC)

texts = ["Warum bin ich so fröhlich?", "Ich liebe Käse.", "Was ist künstliche Intelligenz?"]
response = embedder_openaisdk.get_full_response(texts)

show_pretty_json(response)

In [ ]:
texts = ["Warum bin ich so fröhlich?", "Ich liebe Käse.", "Was ist künstliche Intelligenz?"]
vectors = embedder_openaisdk.get_embeddings(texts)

show_pretty_json(vectors)

### 🏋️ **[ÜBUNG_2.3.01]** Vergleich von Embedding-Werten durch Cosinus Similarity

- *Diese Übung ist Teil von ÜBUNG4. Speichere deine Ergebnisse und Notizen in einem File (Word), füge dann noch die restlichen Übungen dieser Session 2.2 hinzu und lade sie auf Moodle unter "Abgabe Übung 4" bis zum 20.05.25 hoch.*
- Verwende 3 verschiedene Models
- Datenset: Text1, Text2 (ähnlich zu Text1), Text3 (unähnlich zu Text1)
- Pro Model Cosinus Similarity der Vektoren berechnen
- Mit weiterem 2. Datenset probieren (Texte austauschen)
- Mit weiterem 3. Datenset probieren (Texte austauschen)
- Erfahrungen notieren (auch die Vektor-Dimensionsanzahl eines jeden Models)

### 🔓 **[LÖSUNG_2.3.01]** Vergleich von Embeddingwerten von Modellen durch Cosinus Similarity